语言模型本身无法采取行动 - 它们只是输出文本。LangChain 的一个重要用例是创建代理。 代理是使用LLM作为推理引擎来确定要采取哪些操作以及执行操作所需的输入的系统。执行操作后，结果可以反馈到 LLM 中，以确定是否需要更多操作，或者是否可以完成。这通常是通过工具调用来实现的。

在本教程中，我们将构建一个可以与搜索引擎交互的代理。您将能够向该代理提问、观察它调用搜索工具以及与其进行对话。

我们首先需要创建我们想要使用的工具。我们选择的主要工具是Tavily - 一个搜索引擎。我们在 LangChain 中有一个内置工具，可以轻松使用 Tavily 搜索引擎作为工具。

In [6]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv()

# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "llama3.2"
model  = ChatOllama(model=model_name)

# Create the agent
memory = MemorySaver()
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [7]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (e05e54d8-4e0e-4f17-b374-e1ffe5138049)
 Call ID: e05e54d8-4e0e-4f17-b374-e1ffe5138049
  Args:
    query: San Francisco news
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.sfchronicle.com/", "content": "Exclusive Bay Area and San Francisco breaking news, sports, tech, and food and wine coverage, plus enhanced coverage of Giants, 49ers and Warriors."}, {"url": "https://www.sfgate.com/", "content": "Your San Francisco Bay Area local news source plus the latest in sports, culture, weather, food and drink, politics, real estate, Lake Tahoe and California"}]
================================== Ai Message ==================================

Hi Bob! It sounds like you live in Sa

In [8]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (3472f5b6-a648-4afe-b56a-b16ab7c40e6a)
 Call ID: 3472f5b6-a648-4afe-b56a-b16ab7c40e6a
  Args:
    query: San Francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1741156867, 'localtime': '2025-03-04 22:41'}, 'current': {'last_updated_epoch': 1741156200, 'last_updated': '2025-03-04 22:30', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003

简单定义一个calculatot工具

In [33]:
from langchain.tools import Tool
from pydantic import BaseModel, Field


class SimpleCalculatorTool(BaseModel):
    def __init__(self):
        name: str = Field(default="simple_calculator_tool")
        description: str = Field(default="This tool performs basic arithmetic operations (add, subtract, multiply, divide).")
    

    def _run(self, operation: str, x: float, y: float) -> float:
        if operation == "add":
            return x + y
        elif operation == "subtract":
            return x - y
        elif operation == "multiply":
            return x * y
        elif operation == "divide":
            if y == 0:
                raise ValueError("Cannot divide by zero")
            return x / y
        else:
            raise ValueError("Invalid operation")

    def _arun(self, operation: str, x: float, y: float) -> float:
        # 异步执行时可以直接调用同步的 _run 方法
        return self._run(operation, x, y)
    
calculator  = SimpleCalculatorTool()


In [40]:
from langchain_core.tools import tool

@tool
def multiply(a: float, b: float) -> float:
   """Multiply two numbers."""
   return a * b

In [41]:
# Create the agent
memory_calculator = MemorySaver()

tools_calculator= [multiply]

agent_calculator = create_react_agent(model, 
                                      tools_calculator)

In [42]:
for step in agent_calculator.stream(
    {"messages": [HumanMessage(content="计算300的25%")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

计算300的25%
================================== Ai Message ==================================
Tool Calls:
  multiply (0a89a8bf-3a13-4c57-b393-9bf1155754b5)
 Call ID: 0a89a8bf-3a13-4c57-b393-9bf1155754b5
  Args:
    a: 300
    b: 0.25
================================= Tool Message =================================
Name: multiply

75.0
================================== Ai Message ==================================

300 的 25% 是 75.0。
